In [12]:
import re, pathlib, 
import numpy as np
import matplotlib.pyplot as plt

ROOT   = pathlib.Path("")
saves = "pdf" # pdf oder was anderes 
d_whitelist = {12}

In [13]:
plt.style.use('thesis.mplstyle')

# https://regex101.com
regex = re.compile(
    r"d(?P<d>\d+)_h(?P<h>[^_]+)_alpha(?P<alpha>[^_]+)_vt(?P<vt>[^_]+)"
    r"_ls(?P<ls>[^_]+)_sig(?P<sig>[^_]+)_sf(?P<sf>[^_]+)_sg(?P<sg>[^_]+)"
    r"_meth(?P<meth>[^_]+)_seed(?P<seed>\d+)"
)

def parse(fname: str):
    m = regex.search(fname)
    if m is None:
        raise ValueError(f"Unexpected fillename pattern: {fname}")
    groups = m.groupdict()
    return int(groups["d"]), float(groups["ls"]), float(groups["sig"])

records = [] # (d, ls, sig, avg_min, avg_final, avg_step10)

In [14]:
savepath   = pathlib.Path("")   # where the npz live

In [15]:
for f in ROOT.glob("run_data_*.npz"):
    d, ls, sig = parse(f.name)
    if d not in d_whitelist: # skip other lattice sizes
        continue
    data = np.load(f, allow_pickle=False)
    e = data["energy_paths_NLC"]
    avg_min    = e.min(1).mean()
    avg_final  = e[:, -1].mean()
    step10_idx = 10
    avg_step10 = e[:, step10_idx].mean()
    records.append((d, ls, sig, avg_min, avg_final, avg_step10))

In [ ]:
TARGET = {12: -20.064624685045146, 8: -13.385005233150757}

all_vals = np.array([m for r in records for m in r[3:]])
#v_min   = np.nanmin(all_vals)
#v_max   = np.nanmax(all_vals)

v_min   = -2.5
v_max   = 0.1

for d in sorted(d_whitelist):
    sig_vals = sorted({r[2] for r in records if r[0] == d})
    ls_vals  = sorted({r[1] for r in records if r[0] == d})
    sig_to_i = {s: i for i, s in enumerate(sig_vals)}
    ls_to_j  = {l: j for j, l in enumerate(ls_vals)}
    grids = {
        "avg-min":   np.full((len(ls_vals), len(sig_vals)), np.nan),
        "Average after 50 steps": np.full((len(ls_vals), len(sig_vals)), np.nan),
        "Average after 10 steps":np.full((len(ls_vals), len(sig_vals)), np.nan),
    }
    for _, ls, sig, m1, m2, m3 in (r for r in records if r[0] == d):
        i, j = ls_to_j[ls], sig_to_i[sig]
        grids["avg-min"][i, j]    = m1
        grids["Average after 50 steps"][i, j]  = m2
        grids["Average after 10 steps"][i, j] = m3
    for metric, Z in grids.items():
        fig, ax = plt.subplots(figsize=(4, 3))

        im = ax.imshow(np.log10((np.ma.masked_invalid(Z) - TARGET[d])/d), origin="lower", aspect="auto", cmap="viridis", vmin=v_min, vmax=v_max)

        ax.set_xticks(range(len(sig_vals)))
        ax.set_xticklabels(np.round(np.array(sig_vals)**2, 2), rotation=60)
        ax.set_yticks(range(len(ls_vals)))
        ax.set_yticklabels(ls_vals)
        ax.set_xlabel(r"Signal variance $\sigma_f^{2}$")
        ax.set_ylabel(r"Lengthscale $\ell$")

        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label({
            "avg-min":   r"$\log_{10}[E_{\min} - E_{\mathrm{sol}}]$",
            "Average after 50 steps": r"$\log_{10}[E_{\mathrm{final}} - E_{\mathrm{sol}}]$",
            "Average after 10 steps":r"$\log_{10}[E_{\mathrm{step }10} - E_{\mathrm{sol}}]$",
        }[metric])

        #cbar.ax.axhline(TARGET[d]/d, color="red", lw=1.5)

        ax.set_title(f"$d={d}$  — {metric}")
        fig.tight_layout()
        out = savepath / f"{metric}_d{d}.{saves}"
        fig.savefig(out, dpi=600, bbox_inches="tight")
        plt.close(fig)

In [ ]:
import re, pathlib, 
import numpy as np
import matplotlib.pyplot as plt

ROOT   = pathlib.Path("/home/yzimmerm/experiments/results_sweep_2")
saveas = "pdf"
d_whitelist = {8}

plt.style.use('thesis.mplstyle')


# https://regex101.com
regex = re.compile(
    r"d(?P<d>\d+)_h(?P<h>[^_]+)_alpha(?P<alpha>[^_]+)_vt(?P<vt>[^_]+)"
    r"_ls(?P<ls>[^_]+)_sig(?P<sig>[^_]+)_sf(?P<sf>[^_]+)_sg(?P<sg>[^_]+)"
    r"_meth(?P<meth>[^_]+)_seed(?P<seed>\d+)"
)

def parse(fname: str):
    m = regex.search(fname)
    if m is None:
        raise ValueError(f"Unexpected filename pattern: {fname}")
    groups = m.groupdict()
    return int(groups["d"]), float(groups["vt"]), float(groups["sf"])

records = [] # (d, vt, sigma_fg, avg_min, avg_final, avg_step10)
savepath   = pathlib.Path("")
savepath.mkdir(parents=True, exist_ok=True)
for f in ROOT.glob("run_data_*.npz"):
    try:
        d, vt, sigma_fg = parse(f.name)
        if d not in d_whitelist:
            continue
        data = np.load(f, allow_pickle=False)
        e = data["energy_paths_NLC"]
        avg_min    = e.min(1).mean()
        avg_final  = e[:, -1].mean()
        step10_idx = 10
        avg_step10 = e[:, step10_idx].mean()
        records.append((d, vt, sigma_fg, avg_min, avg_final, avg_step10))
    except Exception as err:
        print(f"[SKIP] {f.name}: {err}")

if not records:
    raise RuntimeError("No valid result files found.")

TARGET = {8: -13.385005233150757}

all_vals = np.array([m for r in records for m in r[3:]])
v_min   = -3.5
v_max   = -0.25

for d in sorted(d_whitelist):
    vt_vals  = sorted({r[1] for r in records if r[0] == d})
    sig_vals = sorted({r[2] for r in records if r[0] == d})
    vt_to_i = {v: i for i, v in enumerate(vt_vals)}
    sig_to_j = {s: j for j, s in enumerate(sig_vals)}

    grids = {
        "avg-min":    np.full((len(vt_vals), len(sig_vals)), np.nan),
        "Average after 50 steps":  np.full((len(vt_vals), len(sig_vals)), np.nan),
        "Average after 10 steps": np.full((len(vt_vals), len(sig_vals)), np.nan),
    }

    for _, vt, sig, m1, m2, m3 in (r for r in records if r[0] == d):
        i, j = vt_to_i[vt], sig_to_j[sig]
        grids["avg-min"][i, j]    = m1
        grids["Average after 50 steps"][i, j]  = m2
        grids["Average after 10 steps"][i, j] = m3

    for metric, Z in grids.items():
        fig, ax = plt.subplots(figsize=(3.2, 2.6))

        im = ax.imshow(
            np.log10((np.ma.masked_invalid(Z) - TARGET[d])/d),
            origin="lower", aspect="auto", cmap="viridis", vmin=v_min, vmax=v_max
        )

        ax.set_xticks(range(len(sig_vals)))
        ax.set_xticklabels(sig_vals, rotation=60)
        ax.set_yticks(range(len(vt_vals)))
        ax.set_yticklabels(vt_vals)
        ax.set_xlabel(r"Noise Variance $\sigma_n^2 = \sigma_g^2$")
        ax.set_ylabel(r"Variance threshold $\epsilon$")

        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label({
            "avg-min":   r"$\log_{10}[E_{\min} - E_{\mathrm{sol}}]$",
            "Average after 50 steps": r"$\log_{10}[E_{\mathrm{final}} - E_{\mathrm{sol}}]$",
            "Average after 10 steps":r"$\log_{10}[E_{\mathrm{step }10} - E_{\mathrm{sol}}]$",
        }[metric])

        #cbar.ax.axhline(TARGET[d]/d, color="red", lw=1.5)

        ax.set_title(f"$d={d}$  —  {metric}")
        fig.tight_layout()
        out = savepath / f"{metric}_d{d}_variance.{saveas}"
        fig.savefig(out, dpi=600, bbox_inches="tight")
        plt.close(fig)